In [0]:
pip install ipdb

  Stored in directory: /root/.cache/pip/wheels/59/24/91/695211bd228d40fb22dff0ce3f05ba41ab724ab771736233f3
Successfully built ipdb


In [0]:
import keras
import os
import cv2
import cv2 as cv
import os
import ipdb
import numpy as np
import pandas as pd
import skimage
import tensorflow as tf
from keras import backend as K
import random
import skimage
from skimage.transform import rescale
keras.__version__

Using TensorFlow backend.


'2.2.4'

In [0]:
# -*- coding: utf-8 -*-
"""ResNet152 model for Keras.

# Reference:

- [Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385)

Adapted from code contributed by flyyufelix, mvoelk, BigMoyan, fchollet, adamcasson
"""

import os
import warnings

from keras.layers import Input
from keras.layers import Dense
from keras.layers import Activation
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import GlobalMaxPooling2D
from keras.layers import ZeroPadding2D
from keras.layers import AveragePooling2D
from keras.layers import GlobalAveragePooling2D
from keras.layers import BatchNormalization
from keras.layers import add
from keras.models import Model
import keras.backend as K
from keras.engine.topology import get_source_inputs
from keras.utils import layer_utils
from keras import initializers
from keras.engine import Layer, InputSpec
from keras.utils.data_utils import get_file
from keras_applications.imagenet_utils import _obtain_input_shape

import sys

sys.setrecursionlimit(3000)

WEIGHTS_PATH = 'https://github.com/GKalliatakis/Keras-Application-Zoo/releases/download/0.1/resnet152_weights_tf_dim_ordering_tf_kernels.h5'
WEIGHTS_PATH_NO_TOP = 'https://github.com/GKalliatakis/Keras-Application-Zoo/releases/download/0.1/resnet152_weights_tf_dim_ordering_tf_kernels_notop.h5'


def identity_block(input_tensor, kernel_size, filters, stage, block):
    """The identity_block is the block that has no conv layer at shortcut

    # Arguments
        input_tensor: input tensor
        kernel_size: default 3, the kernel size of middle conv layer at main path
        filters: list of integers, the filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names

    # Returns
        Output tensor for the block.
    """
    eps = 1.1e-5

    if K.image_dim_ordering() == 'tf':
        bn_axis = 3
    else:
        bn_axis = 1

    nb_filter1, nb_filter2, nb_filter3 = filters
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    scale_name_base = 'scale' + str(stage) + block + '_branch'

    x = Conv2D(nb_filter1, (1, 1), name=conv_name_base + '2a', use_bias=False)(input_tensor)
    x = BatchNormalization(epsilon=eps, axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Scale(axis=bn_axis, name=scale_name_base + '2a')(x)
    x = Activation('relu', name=conv_name_base + '2a_relu')(x)

    x = ZeroPadding2D((1, 1), name=conv_name_base + '2b_zeropadding')(x)
    x = Conv2D(nb_filter2, (kernel_size, kernel_size), name=conv_name_base + '2b', use_bias=False)(x)
    x = BatchNormalization(epsilon=eps, axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Scale(axis=bn_axis, name=scale_name_base + '2b')(x)
    x = Activation('relu', name=conv_name_base + '2b_relu')(x)

    x = Conv2D(nb_filter3, (1, 1), name=conv_name_base + '2c', use_bias=False)(x)
    x = BatchNormalization(epsilon=eps, axis=bn_axis, name=bn_name_base + '2c')(x)
    x = Scale(axis=bn_axis, name=scale_name_base + '2c')(x)

    x = add([x, input_tensor], name='res' + str(stage) + block)
    x = Activation('relu', name='res' + str(stage) + block + '_relu')(x)
    return x


def conv_block(input_tensor, kernel_size, filters, stage, block, strides=(2, 2)):
    """conv_block is the block that has a conv layer at shortcut

     # Arguments
        input_tensor: input tensor
        kernel_size: default 3, the kernel size of middle conv layer at main path
        filters: list of integers, the filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names

    # Returns
        Output tensor for the block.
    Note that from stage 3, the first conv layer at main path is with strides=(2,2)
    And the shortcut should have strides=(2,2) as well
    """
    eps = 1.1e-5

    if K.image_dim_ordering() == 'tf':
        bn_axis = 3
    else:
        bn_axis = 1

    nb_filter1, nb_filter2, nb_filter3 = filters
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    scale_name_base = 'scale' + str(stage) + block + '_branch'

    x = Conv2D(nb_filter1, (1, 1), strides=strides, name=conv_name_base + '2a', use_bias=False)(input_tensor)
    x = BatchNormalization(epsilon=eps, axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Scale(axis=bn_axis, name=scale_name_base + '2a')(x)
    x = Activation('relu', name=conv_name_base + '2a_relu')(x)

    x = ZeroPadding2D((1, 1), name=conv_name_base + '2b_zeropadding')(x)
    x = Conv2D(nb_filter2, (kernel_size, kernel_size),
               name=conv_name_base + '2b', use_bias=False)(x)
    x = BatchNormalization(epsilon=eps, axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Scale(axis=bn_axis, name=scale_name_base + '2b')(x)
    x = Activation('relu', name=conv_name_base + '2b_relu')(x)

    x = Conv2D(nb_filter3, (1, 1), name=conv_name_base + '2c', use_bias=False)(x)
    x = BatchNormalization(epsilon=eps, axis=bn_axis, name=bn_name_base + '2c')(x)
    x = Scale(axis=bn_axis, name=scale_name_base + '2c')(x)

    shortcut = Conv2D(nb_filter3, (1, 1), strides=strides,
                      name=conv_name_base + '1', use_bias=False)(input_tensor)
    shortcut = BatchNormalization(epsilon=eps, axis=bn_axis, name=bn_name_base + '1')(shortcut)
    shortcut = Scale(axis=bn_axis, name=scale_name_base + '1')(shortcut)

    x = add([x, shortcut], name='res' + str(stage) + block)
    x = Activation('relu', name='res' + str(stage) + block + '_relu')(x)
    return x


class Scale(Layer):
    """Custom Layer for ResNet used for BatchNormalization.

    Learns a set of weights and biases used for scaling the input data.
    the output consists simply in an element-wise multiplication of the input
    and a sum of a set of constants:

        out = in * gamma + beta,

    where 'gamma' and 'beta' are the weights and biases larned.

    Keyword arguments:
    axis -- integer, axis along which to normalize in mode 0. For instance,
        if your input tensor has shape (samples, channels, rows, cols),
        set axis to 1 to normalize per feature map (channels axis).
    momentum -- momentum in the computation of the exponential average
        of the mean and standard deviation of the data, for
        feature-wise normalization.
    weights -- Initialization weights.
        List of 2 Numpy arrays, with shapes:
        `[(input_shape,), (input_shape,)]`
    beta_init -- name of initialization function for shift parameter
        (see [initializers](../initializers.md)), or alternatively,
        Theano/TensorFlow function to use for weights initialization.
        This parameter is only relevant if you don't pass a `weights` argument.
    gamma_init -- name of initialization function for scale parameter (see
        [initializers](../initializers.md)), or alternatively,
        Theano/TensorFlow function to use for weights initialization.
        This parameter is only relevant if you don't pass a `weights` argument.

    """

    def __init__(self, weights=None, axis=-1, momentum=0.9, beta_init='zero', gamma_init='one', **kwargs):
        self.momentum = momentum
        self.axis = axis
        self.beta_init = initializers.get(beta_init)
        self.gamma_init = initializers.get(gamma_init)
        self.initial_weights = weights
        super(Scale, self).__init__(**kwargs)

    def build(self, input_shape):
        self.input_spec = [InputSpec(shape=input_shape)]
        shape = (int(input_shape[self.axis]),)

        self.gamma = K.variable(self.gamma_init(shape), name='%s_gamma' % self.name)
        self.beta = K.variable(self.beta_init(shape), name='%s_beta' % self.name)
        self.trainable_weights = [self.gamma, self.beta]

        if self.initial_weights is not None:
            self.set_weights(self.initial_weights)
            del self.initial_weights

    def call(self, x, mask=None):
        input_shape = self.input_spec[0].shape
        broadcast_shape = [1] * len(input_shape)
        broadcast_shape[self.axis] = input_shape[self.axis]

        out = K.reshape(self.gamma, broadcast_shape) * x + K.reshape(self.beta, broadcast_shape)
        return out

    def get_config(self):
        config = {"momentum": self.momentum, "axis": self.axis}
        base_config = super(Scale, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))



def ResNet152(include_top=True, weights='imagenet',
              input_tensor=None, input_shape=None,
              large_input=False,
              pooling=None,
              classes=1000):
    """Instantiate the ResNet152 architecture.

    Optionally loads weights pre-trained
    on ImageNet. Note that when using TensorFlow,
    for best performance you should set
    `image_data_format='channels_last'` in your Keras config
    at ~/.keras/keras.json.

    The model and the weights are compatible with both
    TensorFlow and Theano. The data format
    convention used by the model is the one
    specified in your Keras config file.

    # Arguments
        include_top: whether to include the fully-connected
            layer at the top of the network.
        weights: one of `None` (random initialization),
              'imagenet' (pre-training on ImageNet),
              or the path to the weights file to be loaded.
        input_tensor: optional Keras tensor (i.e. output of `layers.Input()`)
            to use as image input for the model.
        input_shape: optional shape tuple, only to be specified
            if `include_top` is False (otherwise the input shape
            has to be `(224, 224, 3)` (with `channels_last` data format)
            or `(3, 224, 224)` (with `channels_first` data format).
            It should have exactly 3 inputs channels,
            and width and height should be no smaller than 197.
            E.g. `(200, 200, 3)` would be one valid value.
        large_input: if True, then the input shape expected will be
            `(448, 448, 3)` (with `channels_last` data format) or
            `(3, 448, 448)` (with `channels_first` data format). (default False)
        pooling: Optional pooling mode for feature extraction
            when `include_top` is `False`.
            - `None` means that the output of the model will be
                the 4D tensor output of the
                last convolutional layer.
            - `avg` means that global average pooling
                will be applied to the output of the
                last convolutional layer, and thus
                the output of the model will be a 2D tensor.
            - `max` means that global max pooling will
                be applied.
        classes: optional number of classes to classify images
            into, only to be specified if `include_top` is True, and
            if no `weights` argument is specified.

    # Returns
        A Keras model instance.

    # Raises
        ValueError: in case of invalid argument for `weights`,
            or invalid input shape.
    """
    if not (weights in {'imagenet', None} or os.path.exists(weights)):
        raise ValueError('The `weights` argument should be either '
                         '`None` (random initialization), `imagenet` '
                         '(pre-training on ImageNet), '
                         'or the path to the weights file to be loaded.')

    if weights == 'imagenet' and include_top and classes != 1000:
        raise ValueError('If using `weights` as imagenet with `include_top`'
                         ' as true, `classes` should be 1000')

    eps = 1.1e-5

    if large_input:
        img_size = 448
    else:
        img_size = 224

    # Determine proper input shape
    input_shape = _obtain_input_shape(input_shape,
                                      default_size=img_size,
                                      min_size=197,
                                      data_format=K.image_data_format(),
                                      require_flatten=include_top)

    if input_tensor is None:
        img_input = Input(shape=input_shape)
    else:
        if not K.is_keras_tensor(input_tensor):
            img_input = Input(tensor=input_tensor, shape=input_shape)
        else:
            img_input = input_tensor

    # handle dimension ordering for different backends
    if K.image_dim_ordering() == 'tf':
        bn_axis = 3
    else:
        bn_axis = 1

    x = ZeroPadding2D((3, 3), name='conv1_zeropadding')(img_input)
    x = Conv2D(64, (7, 7), strides=(2, 2), name='conv1', use_bias=False)(x)
    x = BatchNormalization(epsilon=eps, axis=bn_axis, name='bn_conv1')(x)
    x = Scale(axis=bn_axis, name='scale_conv1')(x)
    x = Activation('relu', name='conv1_relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2), name='pool1')(x)

    x = conv_block(x, 3, [64, 64, 256], stage=2, block='a', strides=(1, 1))
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='b')
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='c')

    x = conv_block(x, 3, [128, 128, 512], stage=3, block='a')
    for i in range(1, 8):
        x = identity_block(x, 3, [128, 128, 512], stage=3, block='b' + str(i))

    x = conv_block(x, 3, [256, 256, 1024], stage=4, block='a')
    for i in range(1, 36):
        x = identity_block(x, 3, [256, 256, 1024], stage=4, block='b' + str(i))

    x = conv_block(x, 3, [512, 512, 2048], stage=5, block='a')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='b')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='c')

    if large_input:
        x = AveragePooling2D((14, 14), name='avg_pool')(x)
    else:
        x = AveragePooling2D((7, 7), name='avg_pool')(x)

    # include classification layer by default, not included for feature extraction
    if include_top:
        x = Flatten()(x)
        x = Dense(classes, activation='softmax', name='fc1000')(x)
    else:
        if pooling == 'avg':
            x = GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            x = GlobalMaxPooling2D()(x)

    # Ensure that the model takes into account
    # any potential predecessors of `input_tensor`.
    if input_tensor is not None:
        inputs = get_source_inputs(input_tensor)
    else:
        inputs = img_input
    # Create model.
    model = Model(inputs, x, name='resnet152')

    # load weights
    if weights == 'imagenet':
        if include_top:
            weights_path = get_file('resnet152_weights_tf_dim_ordering_tf_kernels.h5',
                                    WEIGHTS_PATH)
        else:
            weights_path = get_file('resnet152_weights_tf_dim_ordering_tf_kernels_notop.h5',
                                    WEIGHTS_PATH_NO_TOP)

        model.load_weights(weights_path, by_name=True)
        if K.backend() == 'theano':
            layer_utils.convert_all_kernels_in_model(model)
            if include_top:
                maxpool = model.get_layer(name='avg_pool')
                shape = maxpool.output_shape[1:]
                dense = model.get_layer(name='fc1000')
                layer_utils.convert_dense_weights_data_format(dense, shape, 'channels_first')

        if K.image_data_format() == 'channels_first' and K.backend() == 'tensorflow':
            warnings.warn('You are using the TensorFlow backend, yet you '
                          'are using the Theano '
                          'image data format convention '
                          '(`image_data_format="channels_first"`). '
                          'For best performance, set '
                          '`image_data_format="channels_last"` in '
                          'your Keras config '
                          'at ~/.keras/keras.json.')
    return model

In [0]:
from keras import models
from keras import layers
#cnn = keras.applications.vgg16.VGG16(weights='imagenet', include_top=True, input_shape=(224, 224, 3))
#cnn = ResNet152(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
#keras.applications.resnet.ResNet152
#keras.applications.inception_resnet_v2.InceptionResNetV2 (done)
#keras.applications.resnext.ResNeXt101
#keras.applications.resnet.ResNet50
# cnn = keras.applications.nasnet.NASNetLarge(weights='imagenet', include_top=False, input_shape=(331, 331, 3))

343613440/343610240 [==============================] - 4s 0us/step


In [0]:
#model = models.Sequential()
#model.add(cnn)
#model.add(layers.AveragePooling2D(pool_size=(11, 11), strides=None, padding='valid', data_format=None))
#model.summary()

In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [0]:
def preprocess_frame(image, target_size=(331,331)):
    
    if len(image.shape) == 2:
        image = np.tile(image[:,:,None], 3)
    elif len(image.shape) == 4:
        image = image[:,:,:,0]
    resized_image = rescale(image, (target_size[0] / image.shape[0], target_size[1] / image.shape[1]), preserve_range=True)
    """
    image = skimage.img_as_float(image).astype(np.float32)
    height, width, rgb = image.shape
    if width == height:
        resized_image = cv2.resize(image, (target_height,target_width))

    elif height < width:
        resized_image = cv2.resize(image, (int(width * float(target_height)/height), target_width))
        cropping_length = int((resized_image.shape[1] - target_height) / 2)
        resized_image = resized_image[:,cropping_length:resized_image.shape[1] - cropping_length]

    else:
        resized_image = cv2.resize(image, (target_height, int(height * float(target_width) / width)))
        cropping_length = int((resized_image.shape[0] - target_width) / 2)
        resized_image = resized_image[cropping_length:resized_image.shape[0] - cropping_length,:]

    print(cv2.resize(resized_image, (target_height, target_width)))
    return cv2.resize(resized_image, (target_height, target_width))
    """
    return resized_image

In [0]:
def print_layers(net):
  for layer in net.layers:
    print(layer.name)

In [0]:
def extract_RGB(video_path, video_save_path):
  print("Number of videos to be processed is {}".format(len(os.listdir(video_path))))
  print("Processed files len is {}".format(len(os.listdir(video_save_path))))
  if len(os.listdir(video_path)) == len(os.listdir(video_save_path)):
    print("input equal output do nothing.")
    #return
  num_frames = 80
  videos = os.listdir(video_path)
  videos = filter(lambda x: x.endswith('avi'), videos)
  #out = cnn.get_layer('global_average_pooling2d_1').output
  #gap1 = K.function([cnn.input], [out])
  print("starting")
  for idx, video in enumerate(videos):
          print("{} {}".format(idx, video))

          if os.path.exists( os.path.join(video_save_path, video[:len(video)-4]  + '.npy') ):
              print("Already processed ... ")
              #continue

          video_fullpath = os.path.join(video_path, video)
          try:
              cap  = cv2.VideoCapture( video_fullpath )
          except:
              pass

          frame_count = 0
          frame_list = []

          while True:
              ret, frame = cap.read()
              if ret is False:
                  break

              frame_list.append(frame)
              frame_count += 1

          frame_list = np.array(frame_list)

          if frame_count > 80:
              frame_indices = np.linspace(0, frame_count, num=num_frames, endpoint=False).astype(int)
              frame_list = frame_list[frame_indices]

          cropped_frame_list = np.array(list(map(lambda x: preprocess_frame(x), frame_list)))
          ## MAKE SURE TO RESHAPE ACCORDING TO EACH USED NET.
          feats = cnn.predict(cropped_frame_list)
          print(feats.shape)
          print(feats)
          feats = feats.squeeze()
          #feats = np.array(gap1([cropped_frame_list])[0])
          print(feats.shape)
          print(feats)
          feats = feats.reshape(frame_list.shape[0], 4032)
          return 
          save_full_path = os.path.join(video_save_path, video[:len(video)-4] + '.npy')
          np.save(save_full_path, feats)

In [0]:
video_path = '/content/gdrive/My Drive/Graduation Project/Datasets/MSVD old version/MSR Video Description Corpus/trainVal'
video_save_path = '/content/gdrive/My Drive/Graduation Project/Different Feature Extractor Models/graduationFeatures/msvd1/shortPreprocessWithAmrFix/NASNet (4032)/trainVal'
extract_RGB(video_path, video_save_path)

Number of videos to be processed is 1577
Processed files len is 1577
input equal output do nothing.
starting
0 0SMOK2ql7Pg_5_12.avi
Already processed ... 


/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:23: UserWarning: The default multichannel argument (None) is deprecated.  Please specify either True or False explicitly.  multichannel will default to False starting with release 0.16.
  warn('The default multichannel argument (None) is deprecated.  Please '


(80, 11, 11, 4032)
[[[[0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
    0.0000000e+00 0.0000000e+00]
   [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
    0.0000000e+00 0.0000000e+00]
   [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
    0.0000000e+00 0.0000000e+00]
   ...
   [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
    0.0000000e+00 0.0000000e+00]
   [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
    0.0000000e+00 0.0000000e+00]
   [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
    0.0000000e+00 0.0000000e+00]]

  [[0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
    0.0000000e+00 0.0000000e+00]
   [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
    0.0000000e+00 0.0000000e+00]
   [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
    0.0000000e+00 0.0000000e+00]
   ...
   [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
    0.0000000e+00 0.0000000e

ValueError: ignored

In [0]:
video_path = '/content/gdrive/My Drive/Graduation Project/Datasets/MSVD old version/MSR Video Description Corpus/test'
video_save_path = '/content/gdrive/My Drive/Graduation Project/Different Feature Extractor Models/graduationFeatures/msvd1/shortPreprocessWithAmrFix/NASNet (4032)/test'
extract_RGB(video_path, video_save_path)

In [0]:
print_layers(cnn)

input_1
stem_conv1
stem_bn1
activation_1
reduction_conv_1_stem_1
reduction_bn_1_stem_1
activation_2
activation_4
separable_conv_1_pad_reduction_left1_stem_1
separable_conv_1_pad_reduction_right1_stem_1
separable_conv_1_reduction_left1_stem_1
separable_conv_1_reduction_right1_stem_1
separable_conv_1_bn_reduction_left1_stem_1
separable_conv_1_bn_reduction_right1_stem_1
activation_3
activation_5
separable_conv_2_reduction_left1_stem_1
separable_conv_2_reduction_right1_stem_1
activation_6
separable_conv_2_bn_reduction_left1_stem_1
separable_conv_2_bn_reduction_right1_stem_1
separable_conv_1_pad_reduction_right2_stem_1
activation_8
reduction_add_1_stem_1
separable_conv_1_reduction_right2_stem_1
separable_conv_1_pad_reduction_right3_stem_1
activation_10
separable_conv_1_bn_reduction_right2_stem_1
separable_conv_1_reduction_right3_stem_1
separable_conv_1_reduction_left4_stem_1
activation_7
separable_conv_1_bn_reduction_right3_stem_1
separable_conv_1_bn_reduction_left4_stem_1
reduction_pad_1_s

In [0]:
print_layers(cnn)

input_2
stem_conv1
stem_bn1
activation_94
reduction_conv_1_stem_1
reduction_bn_1_stem_1
activation_95
activation_97
separable_conv_1_pad_reduction_left1_stem_1
separable_conv_1_pad_reduction_right1_stem_1
separable_conv_1_reduction_left1_stem_1
separable_conv_1_reduction_right1_stem_1
separable_conv_1_bn_reduction_left1_stem_1
separable_conv_1_bn_reduction_right1_stem_1
activation_96
activation_98
separable_conv_2_reduction_left1_stem_1
separable_conv_2_reduction_right1_stem_1
activation_99
separable_conv_2_bn_reduction_left1_stem_1
separable_conv_2_bn_reduction_right1_stem_1
separable_conv_1_pad_reduction_right2_stem_1
activation_101
reduction_add_1_stem_1
separable_conv_1_reduction_right2_stem_1
separable_conv_1_pad_reduction_right3_stem_1
activation_103
separable_conv_1_bn_reduction_right2_stem_1
separable_conv_1_reduction_right3_stem_1
separable_conv_1_reduction_left4_stem_1
activation_100
separable_conv_1_bn_reduction_right3_stem_1
separable_conv_1_bn_reduction_left4_stem_1
reduct

In [0]:
video_path = '/content/gdrive/My Drive/Graduation Project/Datasets/MSVD old version/MSR Video Description Corpus/trainVal'
video_save_path = '/content/gdrive/My Drive/Graduation Project/Different Feature Extractor Models/graduationFeatures/msvd1/shortPreprocessWithAmrFix/NASNet (4032)/trainVal'
extract_RGB(video_path, video_save_path)

Number of videos to be processed is 1577
Number of videos to be processed is 1577
input equal output do nothing.
starting
0 0SMOK2ql7Pg_5_12.avi
Already processed ... 
[[[130 133 138]
  [130 133 138]
  [130 133 138]
  ...
  [116 119 119]
  [116 119 119]
  [113 116 116]]

 [[131 134 139]
  [131 134 139]
  [131 134 139]
  ...
  [118 121 121]
  [115 118 118]
  [112 115 115]]

 [[133 136 141]
  [133 136 141]
  [133 136 141]
  ...
  [120 123 123]
  [115 118 118]
  [108 111 111]]

 ...

 [[ 89  97 106]
  [ 84  92 101]
  [ 79  87  96]
  ...
  [ 36  42  51]
  [ 34  40  49]
  [ 34  40  49]]

 [[ 93  95 102]
  [ 90  92  99]
  [ 83  88  94]
  ...
  [ 37  43  52]
  [ 36  42  51]
  [ 36  42  51]]

 [[ 26  28  35]
  [ 26  28  35]
  [ 18  23  29]
  ...
  [  0   3  12]
  [  0   3  12]
  [  0   3  12]]]
[[[129.04380665 132.04380665 137.04380665]
  [129.94944597 132.94944597 137.94944597]
  [130.04380665 133.04380665 138.04380665]
  ...
  [116.59701217 119.59701217 119.59701217]
  [115.08066511 118.0806

/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:23: UserWarning: The default multichannel argument (None) is deprecated.  Please specify either True or False explicitly.  multichannel will default to False starting with release 0.16.
  warn('The default multichannel argument (None) is deprecated.  Please '


[[[122.         125.         130.        ]
  [122.94712991 125.94712991 130.94712991]
  [123.57854985 126.57854985 131.57854985]
  ...
  [113.57854985 118.57854985 118.57854985]
  [112.89425982 117.89425982 117.89425982]
  [110.31570997 115.31570997 115.31570997]]

 [[122.04149059 125.04149059 130.04149059]
  [122.95407809 125.95407809 130.95407809]
  [123.6339368  126.6339368  131.6339368 ]
  ...
  [113.68932376 118.68932376 118.68932376]
  [113.28157145 118.28157145 118.28157145]
  [110.57854985 115.57854985 115.57854985]]

 [[122.46559223 125.46559223 130.46559223]
  [123.20745293 126.20745293 131.20745293]
  [124.         127.         132.        ]
  ...
  [114.85951662 119.85951662 119.85951662]
  [116.2910365  121.2910365  121.2910365 ]
  [112.82292741 117.82292741 117.82292741]]

 ...

 [[ 85.3429003   96.3429003  104.3429003 ]
  [ 85.3429003   96.3429003  104.3429003 ]
  [ 85.3429003   96.3429003  104.3429003 ]
  ...
  [ 37.78096677  46.56193353  52.56193353]
  [ 37.52064375  4

In [0]:
"""
import os
from shutil import copyfile
source = os.listdir(video_path)
newDest = "/content/gdrive/My Drive/Graduation Project/Different Feature Extractor Models/graduationFeatures/vgg16-(4096)WithFixes/trainVal2"
for f in source:
  f = f[:len(f) - 4] + ".npy"
  print(f)
  copyfile(os.path.join(video_save_path, f), os.path.join(newDest, f))
"""

In [0]:
print(len(os.listdir(newDest)))

7010


In [0]:
def extract_optical_flow(video_path, video_save_path):
  videos = os.listdir(video_path)
  videos = filter(lambda x: x.endswith('mp4'), videos)
  for idx, video in enumerate(videos):
    print("{} {}".format(idx, video))

    if os.path.exists( os.path.join(video_save_path, video) ):
        print("Already processed ... ")
        continue
    frame_list = []
    frame_count = 1
    video_fullpath = os.path.join(video_path, video)
    cap = cv.VideoCapture(video_fullpath)
    ret, frame1 = cap.read()
    prvs = cv.cvtColor(frame1,cv.COLOR_BGR2GRAY)
    flow = cv.calcOpticalFlowFarneback(prvs, prvs ,None, 0.5, 3, 15, 3, 5, 1.2, 0)
    frame_list.append(flow)
    while(1):
      ret, frame2 = cap.read()
      if not ret:
        break
      nex = cv.cvtColor(frame2,cv.COLOR_BGR2GRAY)
      flow = cv.calcOpticalFlowFarneback(prvs, nex ,None, 0.5, 3, 15, 3, 5, 1.2, 0)
      prvs = nex
      frame_list.append(flow)
      frame_count += 1
      print(flow.shape)

    frame_list = np.array(frame_list)

    if frame_count > 80:
        frame_indices = np.linspace(0, frame_count, num=num_frames, endpoint=False).astype(int)
        frame_list = frame_list[frame_indices]
    cap.release()
    save_full_path = os.path.join(video_save_path, video + '.npy')
    
    #np.save(save_full_path, feats)

In [0]:
video_path = '/content/gdrive/My Drive/MSVD/TrainValVideo'
video_save_path = '/content/gdrive/My Drive/graduationFeatures/opticalFlow/TrainVal'
#extract_optical_flow(video_path, video_save_path)

In [0]:
# temporal conv neural network
# attention model.

In [0]:
cnn = keras.applications.nasnet.NASNetLarge(weights='imagenet', include_top=True, input_shape=(331, 331, 3))
model = models.Sequential()
model.add(cnn)
model.add(layers.AveragePooling2D(pool_size=(7, 7), strides=None, padding='valid', data_format=None))

In [0]:
def download_video(link):
    DEFAULT_DIR = '/content/videos'
    os.system('rm -r {}'.format(DEFAULT_DIR))
    os.system('mkdir {}'.format(DEFAULT_DIR))
    yt = YouTube(link)
    yt.streams.first().download(DEFAULT_DIR)
    return os.path.join(DEFAULT_DIR, yt.title + ".mp4")
import os
os.system('pip install pytube')
import sys
stdout = sys.stdout
from pytube import YouTube
sys.stdout = stdout

In [0]:
download_video("https://www.youtube.com/watch?v=ELeMaP8EPAA&fbclid=IwAR0Vs-8FX1EnsH4eabYvFwMqMFuLIEL-5H4zVeWMi7AnkIv1MmXfOn5XTJ4")

'/content/videos/ONCE UPON A TIME IN HOLLYWOOD - Official Trailer (HD).mp4'

In [0]:
os.path.exists("/content/videos/ONCE UPON A TIME IN HOLLYWOOD - Official Trailer (HD).mp4")

True

In [0]:
download_video("https://i.ytimg.com/an_webp/74vqjhrkeAw/mqdefault_6s.webp?du=3000&sqp=CPSbkucF&rs=AOn4CLByb6n-MVPTEo970D5IaaHLgQCB1A")

'/content/videos/الحلقة الاولى من " يانا يانتى " احمد حلمى - منى زكي / Yana Yanty - Ahmed Helmy & Mona Zaki.mp4'